In [2]:
%pip install requests feedparser
%pip install pandas
%pip install googletrans==4.0.0-rc1
%pip install influxdb-client


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Cell 1 - imports + config (DB-only)
import os
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timezone

# InfluxDB (read from env if you want)
INFLUX_URL = os.getenv("INFLUX_URL", "http://localhost:8086")
INFLUX_TOKEN = os.getenv("INFLUX_TOKEN", "")
INFLUX_ORG = os.getenv("INFLUX_ORG", "")
INFLUX_BUCKET = os.getenv("INFLUX_BUCKET", "")

FEED_URL = "https://rss.orf.at/news.xml"

TARGET_OEWA = "urn:oewa:RedCont:Politik/PolitikAusland"
USER_AGENT = "orf-rss-tracker/1.0 (+local notebook)"


In [4]:
# Cell 2 - fetch XML
def fetch_feed_xml(url: str, timeout: int = 20) -> str:
    r = requests.get(url, timeout=timeout, headers={"User-Agent": USER_AGENT})
    r.raise_for_status()
    return r.text

xml_text = fetch_feed_xml(FEED_URL)
len(xml_text), xml_text[:200]


(14846,
 '<?xml version="1.0" encoding="UTF-8"?>\n<rdf:RDF\n  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n  xmlns:dc="http://purl.org/dc/elements/1.1/"\n  xmlns:sy="http://purl.org/rss/1.0/modules/synd')

In [5]:
# Cell 3 - detect namespaces robustly (so you don't have to guess)
import io

def detect_namespaces(xml_text: str) -> dict:
    ns = {}
    for event, elem in ET.iterparse(io.StringIO(xml_text), events=("start-ns",)):
        prefix, uri = elem
        ns[prefix if prefix is not None else ""] = uri
    return ns

NS = detect_namespaces(xml_text)
NS


{'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'dc': 'http://purl.org/dc/elements/1.1/',
 'sy': 'http://purl.org/rss/1.0/modules/syndication/',
 'orfon': 'http://rss.orf.at/1.0/',
 '': 'http://purl.org/rss/1.0/'}

In [6]:
# Cell 4 - parse + sanity checks (THIS will show why your old code returned 0)
root = ET.fromstring(xml_text)

rss_ns = NS.get("rss", "http://purl.org/rss/1.0/")  # ORF uses RSS 1.0
items = root.findall(".//{%s}item" % rss_ns)

root.tag, len(items)


('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF', 24)

In [7]:
# Cell 5 - helper: load already-seen usids (dedupe) from InfluxDB (DB-only)
from influx_io import get_client, INFLUX_BUCKET, INFLUX_ORG

def load_seen_usids_from_influx(lookback: str = "30d") -> set[str]:
    """
    Pull distinct usid tag values stored in measurement 'orf_article'
    within a recent lookback window.
    """
    with get_client() as client:
        query_api = client.query_api()

        flux = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: -{lookback})
  |> filter(fn: (r) => r._measurement == "orf_article")
  |> keep(columns: ["usid"])
  |> distinct(column: "usid")
'''

        tables = query_api.query(flux, org=INFLUX_ORG)

    seen = set()
    for table in tables:
        for record in table.records:
            val = record.values.get("usid")
            if val:
                seen.add(str(val))
    return seen

seen_usids = load_seen_usids_from_influx("30d")
len(seen_usids)


0

In [8]:
# Cell 6 - parse items + filter by oewaCategory
def text_of(el):
    return el.text.strip() if el is not None and el.text else None

def parse_filtered_items(root: ET.Element, ns: dict) -> list[dict]:
    rss_ns = ns.get("rss", "http://purl.org/rss/1.0/")
    rdf_ns = ns.get("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    dc_ns  = ns.get("dc",  "http://purl.org/dc/elements/1.1/")
    orf_ns = ns.get("orfon")  # must exist in feed; we'll rely on detected value

    if not orf_ns:
        raise RuntimeError("Could not detect 'orfon' namespace in the feed. Check NS dict output.")

    out = []
    for item in root.findall(".//{%s}item" % rss_ns):
        # orfon:oewaCategory rdf:resource="..."
        cat_el = item.find("{%s}oewaCategory" % orf_ns)
        if cat_el is None:
            continue

        cat_val = cat_el.attrib.get("{%s}resource" % rdf_ns)
        if cat_val != TARGET_OEWA:
            continue

        title_el = item.find("{%s}title" % rss_ns)
        link_el  = item.find("{%s}link" % rss_ns)
        date_el  = item.find("{%s}date" % dc_ns)
        usid_el  = item.find("{%s}usid" % orf_ns)

        out.append({
            "usid": text_of(usid_el),
            "date": text_of(date_el),
            "link": text_of(link_el),
            "title": text_of(title_el),
            "oewaCategory": cat_val,
            "fetched_at_utc": datetime.now(timezone.utc).isoformat(),
        })

    return out

filtered_items = parse_filtered_items(root, NS)
len(filtered_items), filtered_items[:2]


(10,
 [{'usid': 'news:3416944',
   'date': '2026-01-12T18:16:58+01:00',
   'link': 'https://orf.at/stories/3416944/',
   'title': 'Fünf Jahre Haft für Georgiens Ex-Regierungschef',
   'oewaCategory': 'urn:oewa:RedCont:Politik/PolitikAusland',
   'fetched_at_utc': '2026-01-12T17:21:38.929863+00:00'},
  {'usid': 'news:3416938',
   'date': '2026-01-12T16:51:58+01:00',
   'link': 'https://orf.at/stories/3416938/',
   'title': 'Iran bestellt Botschafter mehrerer Staaten ein',
   'oewaCategory': 'urn:oewa:RedCont:Politik/PolitikAusland',
   'fetched_at_utc': '2026-01-12T17:21:38.929863+00:00'}])

In [9]:
# Cell 7 - (optional) debug: what categories exist + counts
from collections import Counter

def category_counts(root: ET.Element, ns: dict) -> Counter:
    rss_ns = ns.get("rss", "http://purl.org/rss/1.0/")
    rdf_ns = ns.get("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    orf_ns = ns.get("orfon")
    c = Counter()
    for item in root.findall(".//{%s}item" % rss_ns):
        cat_el = item.find("{%s}oewaCategory" % orf_ns) if orf_ns else None
        if cat_el is None:
            continue
        cat_val = cat_el.attrib.get("{%s}resource" % rdf_ns)
        if cat_val:
            c[cat_val] += 1
    return c

counts = category_counts(root, NS)
counts.most_common(10)


[('urn:oewa:RedCont:Politik/PolitikAusland', 10),
 ('urn:oewa:RedCont:Nachrichten/Chronik', 4),
 ('urn:oewa:RedCont:Wirtschaft/Wirtschaftspolitik', 3),
 ('urn:oewa:RedCont:ComputerUndTechnik/ComputerUndTechnikUeberblick', 2),
 ('urn:oewa:RedCont:Politik/PolitikInland', 2),
 ('urn:oewa:RedCont:KulturUndFreizeit/Literatur', 1),
 ('urn:oewa:RedCont:Wirtschaft/Unternehmensberichterstattung', 1),
 ('urn:oewa:RedCont:KulturUndFreizeit/KulturUeberblick', 1)]

In [10]:
# Cell 8 - DB-only: keep only new items + write to InfluxDB
from influx_io import write_orf_articles

new_items = [it for it in filtered_items if it.get("usid") and it["usid"] not in seen_usids]

written = write_orf_articles(new_items)
print("Wrote ORF articles to InfluxDB:", written)

# update local seen set (so rerunning later cells in this same session doesn't re-write)
for it in new_items:
    seen_usids.add(it["usid"])

len(new_items)


Wrote ORF articles to InfluxDB: 10


10

In [11]:
# Cell 9 - show latest rows quickly (DB-only, from current run)
import pandas as pd

df = pd.DataFrame(new_items)
df.tail(10)


,usid,date,link,title,oewaCategory,fetched_at_utc
0,news:3416944,2026-01-12T18:16:58+01:00,https://orf.at/stories/3416944/,Fünf Jahre Haft für Georgiens Ex-Regierungschef,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
1,news:3416938,2026-01-12T16:51:58+01:00,https://orf.at/stories/3416938/,Iran bestellt Botschafter mehrerer Staaten ein,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
2,news:3416926,2026-01-12T15:21:56+01:00,https://orf.at/stories/3416926/,116 politische Gefangene in Venezuela freigela...,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
3,news:3416920,2026-01-12T15:16:04+01:00,https://orf.at/stories/3416920/,Pläne zu Mindestpreisen für Elektroautos aus C...,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
4,news:3416918,2026-01-12T14:58:11+01:00,https://orf.at/stories/3416918/,Grönland-Debatte: NATO arbeitet an „nächsten S...,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
5,news:3416898,2026-01-12T14:13:51+01:00,https://orf.at/stories/3416898/,Was die Proteste für Regime gefährlich macht,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
6,news:3416902,2026-01-12T12:36:41+01:00,https://orf.at/stories/3416902/,GB will ballistische Rakete für Ukraine entwic...,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
7,news:3416895,2026-01-12T11:29:05+01:00,https://orf.at/stories/3416895/,Völkermordprozess gegen Myanmar in Den Haag an...,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
8,news:3416892,2026-01-12T11:05:25+01:00,https://orf.at/stories/3416892/,35.000 Haushalte um Odessa ohne Strom,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00
9,news:3416889,2026-01-12T09:58:25+01:00,https://orf.at/stories/3416889/,Trump: „Wir bekommen Grönland“,urn:oewa:RedCont:Politik/PolitikAusland,2026-01-12T17:21:38.929863+00:00


Reddit

In [12]:
# =========================
# RUN SCRIPT (full)
# =========================
import os
import re
import time
from datetime import datetime, timezone

import pandas as pd
import requests
from googletrans import Translator

# df already exists:
# df = pd.read_csv(CSV_PATH)
df = pd.DataFrame(filtered_items).head(2)


# =========================
# CONFIG
# =========================
OUT_CSV = "reddit_posts_minimal.csv"
USER_AGENT = "orf-reddit-minimal/1.0"

TITLE_COL = "title"
USID_COL = "usid"

# --- Rate limiting ---
REQUEST_SLEEP = 2.0      # <- normal sleep between EVERY reddit request
BACKOFF_SLEEP = 90       # <- sleep when 429 happens

# --- groups / matching ---
MAX_KEYWORDS_DE = 20          # extract more DE keywords from title
ADD_BIGRAM_GROUPS = True      # add phrase/bigram groups to reach ~7+ groups
MAX_BIGRAMS = 10              # cap bigram groups per article

TARGET_GROUPS = 7             # try to use 7 groups
MIN_GROUP_MATCHES = 3         # must hit at least 3 groups
MAX_POST_WORDS = 300          # only keep posts with <= 300 words in checked text

CHECK_TEXT_MODE = "title+selftext"  # or "selftext"

# --- reddit search (ranked) ---
MAX_PAGES_PER_SOURCE = 10
LIMIT_PER_PAGE = 100
SORT_MODE = "relevance"

# --- reddit subreddit feeds (/new.json) ---
USE_SUB_NEW_FEEDS = True
NEW_FEED_PAGES_PER_SUB = 8
NEW_FEED_LIMIT = 100

# --- where to search ---
SUBREDDITS = ["politics", "austria", "europe", "worldnews", "news"]
SEARCH_GLOBAL_TOO = True
RESTRICT_TO_SUBS_ONLY = False

# If you delete the CSV after each run, keep False
DEDUPE_WITH_EXISTING_CSV = False

# =========================
# INIT
# =========================
session = requests.Session()
session.headers.update({"User-Agent": USER_AGENT})
translator = Translator()

STOPWORDS = {
    # DE
    "der","die","das","und","oder","nicht","nur","auch","mit","von","für","über","unter","nach","vor",
    "ein","eine","einer","eines","dem","den","des","im","in","am","an","auf","aus","bei","zum","zur",
    "ist","sind","war","waren","wird","werden","hat","haben","kann","können","muss","müssen",
    # EN
    "the","and","or","not","only","also","with","from","for","about","this","that","these","those",
    "is","are","was","were","be","been","being","has","have","had","can","could","must","will","may",
    "of","by","to","in","on","at"
}

WORD_RE = re.compile(r"\b[\wÄÖÜäöüß]+\b", flags=re.UNICODE)


# =========================
# HELPERS
# =========================
def safe_str(x) -> str:
    if pd.isna(x):
        return ""
    return str(x)


def words_list(text: str) -> list[str]:
    return WORD_RE.findall((text or "").strip())


def tokenize_list(text: str) -> list[str]:
    """Ordered tokens, lowercased, stopwords removed, len>=3."""
    text = (text or "").lower()
    text = re.sub(r"[^0-9a-zäöüß]+", " ", text, flags=re.IGNORECASE)
    toks = []
    for t in text.split():
        if len(t) >= 3 and t not in STOPWORDS:
            toks.append(t)
    return toks


def tokenize_set(text: str) -> set[str]:
    return set(tokenize_list(text))


def normalize_term_variants(term: str) -> set[str]:
    """
    Lightweight variants:
    - lower
    - naive plural/singular + DE endings trimming
    """
    t = (term or "").strip().lower()
    if not t:
        return set()
    vars_ = {t}

    # EN plural -> singular heuristic
    if t.endswith("s") and len(t) > 3:
        vars_.add(t[:-1])

    # DE common endings heuristic
    for suf in ("en", "er", "e", "n", "s"):
        if t.endswith(suf) and len(t) > len(suf) + 2:
            vars_.add(t[: -len(suf)])

    return {v for v in vars_ if len(v) >= 3 and v not in STOPWORDS}


def build_keywords_de(title: str) -> list[str]:
    """
    Deterministic DE keywords:
    - unique tokens
    - prefer longer tokens
    """
    toks = tokenize_list(title)
    uniq = []
    seen = set()
    for t in toks:
        if t not in seen:
            uniq.append(t)
            seen.add(t)
    uniq_sorted = sorted(uniq, key=lambda x: (-len(x), x))
    return uniq_sorted[:MAX_KEYWORDS_DE]


def translate_text_de_to_en(text: str) -> str | None:
    """Best-effort translation; returns None on failure/empty."""
    try:
        t = translator.translate(text, src="de", dest="en").text
        t = (t or "").strip().lower()
        return t if t else None
    except Exception:
        return None


def translate_keywords_to_en(keywords_de: list[str]) -> dict[str, str]:
    """Mapping {de_kw: en_kw} best-effort."""
    m = {}
    for kw in keywords_de:
        t = translate_text_de_to_en(kw)
        if t:
            m[kw] = t
    return m


def build_bigram_phrases(title: str) -> list[str]:
    """
    Build phrase candidates from title tokens in order.
    """
    toks = tokenize_list(title)
    bigrams = []
    for i in range(len(toks) - 1):
        bigrams.append(f"{toks[i]} {toks[i+1]}")
    bigrams = sorted(list(dict.fromkeys(bigrams)), key=lambda x: (-len(x), x))
    return bigrams[:MAX_BIGRAMS]


def build_keyword_groups_from_title(title: str) -> tuple[list[set[str]], dict[str, str], list[str]]:
    """
    Returns:
      - groups (list[set[str]])  : synonym groups (unigrams + optional bigrams)
      - de_to_en_map (dict)      : mapping for unigrams and bigrams that were translated
      - debug_reps (list[str])   : representative term per group (for logging)
    """
    keywords_de = build_keywords_de(title)
    de_to_en = translate_keywords_to_en(keywords_de)

    groups: list[set[str]] = []

    # unigram groups
    for de_kw in keywords_de:
        g = set()
        g |= normalize_term_variants(de_kw)
        en_kw = de_to_en.get(de_kw)
        if en_kw:
            g |= normalize_term_variants(en_kw)
        if g:
            groups.append(g)

    # bigram groups
    if ADD_BIGRAM_GROUPS:
        for bg_de in build_bigram_phrases(title):
            g = set()
            g.add(bg_de.lower())
            for tok in tokenize_list(bg_de):
                g |= normalize_term_variants(tok)

            bg_en = translate_text_de_to_en(bg_de)
            if bg_en:
                de_to_en[bg_de] = bg_en
                g.add(bg_en.lower())
                for tok in tokenize_list(bg_en):
                    g |= normalize_term_variants(tok)

            groups.append(g)

    # dedupe identical groups
    uniq = []
    seen = set()
    for g in groups:
        key = tuple(sorted(g))
        if key not in seen:
            uniq.append(g)
            seen.add(key)
    groups = uniq

    # sort groups by specificity (longest representative)
    scored = []
    for g in groups:
        rep = sorted(g, key=lambda x: (-len(x), x))[0]
        scored.append((len(rep), rep, g))
    scored.sort(reverse=True)

    picked = [g for _, _, g in scored[:TARGET_GROUPS]]
    reps = [rep for _, rep, _ in scored[:TARGET_GROUPS]]

    return picked, de_to_en, reps


def group_hit_count(tokens: set[str], groups: list[set[str]]) -> int:
    return sum(1 for g in groups if (tokens & g))


def get_check_text(post: dict) -> str:
    """
    Include URL as extra matchable text (important for link posts).
    """
    title = safe_str(post.get("title", ""))
    selftext = safe_str(post.get("selftext", ""))
    url = safe_str(post.get("url", ""))

    base = selftext if CHECK_TEXT_MODE == "selftext" else f"{title} {selftext}".strip()
    return f"{base} {url}".strip()


# =========================
# REQUEST WRAPPER (rate limited)
# =========================
def reddit_get(url: str, *, params: dict | None = None, timeout: int = 30):
    """
    Always sleeps REQUEST_SLEEP after each request.
    On 429, sleeps BACKOFF_SLEEP and retries once.
    Returns response or None if still 429/failed.
    """
    try:
        r = session.get(url, params=params, timeout=timeout)
        time.sleep(REQUEST_SLEEP)

        if r.status_code == 429:
            print("⚠️ 429 rate limit. Backing off...")
            time.sleep(BACKOFF_SLEEP)
            r = session.get(url, params=params, timeout=timeout)
            time.sleep(REQUEST_SLEEP)

        if r.status_code == 429:
            print("⚠️ 429 again. Skipping this request.")
            return None

        r.raise_for_status()
        return r
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return None


# =========================
# REDDIT FETCHERS
# =========================
def reddit_search(query: str, subreddit: str | None = None):
    """Search either globally or within a subreddit."""
    if subreddit:
        base = f"https://www.reddit.com/r/{subreddit}/search.json"
    else:
        base = "https://www.reddit.com/search.json"

    after = None
    for _ in range(MAX_PAGES_PER_SOURCE):
        params = {
            "q": query,
            "sort": SORT_MODE,
            "limit": LIMIT_PER_PAGE,
            "syntax": "lucene",
        }
        if subreddit:
            params["restrict_sr"] = 1
        if after:
            params["after"] = after

        r = reddit_get(base, params=params)
        if r is None:
            return

        data = r.json().get("data", {})
        children = data.get("children", [])
        if not children:
            return

        for ch in children:
            yield ch.get("data", {})

        after = data.get("after")
        if not after:
            return


def iter_posts_for_query(query: str):
    """Global + subreddit search; dedupe across sources by reddit_id."""
    seen = set()

    if SEARCH_GLOBAL_TOO and not RESTRICT_TO_SUBS_ONLY:
        for p in reddit_search(query, subreddit=None):
            rid = safe_str(p.get("id"))
            if rid and rid not in seen:
                seen.add(rid)
                yield p

    for sub in SUBREDDITS:
        for p in reddit_search(query, subreddit=sub):
            rid = safe_str(p.get("id"))
            if rid and rid not in seen:
                seen.add(rid)
                yield p


def subreddit_new_feed(subreddit: str, pages: int = NEW_FEED_PAGES_PER_SUB, limit: int = NEW_FEED_LIMIT):
    """Iterate newest posts from a subreddit feed (bypasses search limits)."""
    base = f"https://www.reddit.com/r/{subreddit}/new.json"
    after = None

    for _ in range(pages):
        params = {"limit": limit}
        if after:
            params["after"] = after

        r = reddit_get(base, params=params)
        if r is None:
            return

        data = r.json().get("data", {})
        children = data.get("children", [])
        if not children:
            return

        for ch in children:
            yield ch.get("data", {})

        after = data.get("after")
        if not after:
            return


def prefetch_new_feeds(subreddits: list[str]) -> dict[str, list[dict]]:
    """Prefetch new posts once per run so each article can filter locally."""
    cache: dict[str, list[dict]] = {}
    for sub in subreddits:
        posts = []
        seen = set()
        try:
            for p in subreddit_new_feed(sub):
                rid = safe_str(p.get("id"))
                if rid and rid not in seen:
                    seen.add(rid)
                    posts.append(p)
        except Exception as e:
            print(f"Prefetch error for r/{sub}: {e}")
        cache[sub] = posts
        print(f"Prefetched r/{sub} new: {len(posts)} posts")
    return cache


# =========================
# OPTIONAL DEDUPE WITH EXISTING CSV
# =========================
existing_pairs = set()
if DEDUPE_WITH_EXISTING_CSV and os.path.exists(OUT_CSV):
    existing = pd.read_csv(OUT_CSV, encoding="utf-8")
    if {"article_usid", "reddit_id"}.issubset(existing.columns):
        existing_pairs = set(zip(existing["article_usid"].astype(str), existing["reddit_id"].astype(str)))


# =========================
# PREFETCH /new.json POSTS
# =========================
new_feed_cache = {}
if USE_SUB_NEW_FEEDS:
    new_feed_cache = prefetch_new_feeds(SUBREDDITS)


# =========================
# MAIN
# =========================
rows = []

for idx, r in df.iterrows():
    article_usid = safe_str(r.get(USID_COL))
    article_title = safe_str(r.get(TITLE_COL))
    if not article_usid or not article_title:
        continue

    groups, de_to_en_map, group_reps = build_keyword_groups_from_title(article_title)
    if len(groups) < MIN_GROUP_MATCHES:
        print(f"[{idx+1}/{len(df)}] {article_usid} skipped (only {len(groups)} groups)")
        continue

    # Build DE & EN queries
    reps = group_reps[:]
    reps_de = [t for t in reps if (re.search(r"[äöüß]", t) or t in tokenize_set(article_title))]
    if len(reps_de) < 3:
        reps_de = reps[:]
    reps_de = reps_de[:TARGET_GROUPS]

    en_candidates = sorted({v for v in de_to_en_map.values() if v}, key=lambda x: (-len(x), x))
    reps_en = en_candidates[:TARGET_GROUPS] if len(en_candidates) >= 3 else reps[:TARGET_GROUPS]

    query_de = "(" + " OR ".join(f"\"{k}\"" for k in reps_de) + ")"
    query_en = "(" + " OR ".join(f"\"{k}\"" for k in reps_en) + ")"

    candidates: dict[str, tuple[dict, str]] = {}  # rid -> (post, source)

    def add_candidates_from_iter(it, source: str):
        for p in it:
            rid = safe_str(p.get("id"))
            if not rid:
                continue
            if rid not in candidates:
                candidates[rid] = (p, source)

    # searches
    add_candidates_from_iter(iter_posts_for_query(query_de), "search_de")
    add_candidates_from_iter(iter_posts_for_query(query_en), "search_en")

    # new feeds
    if USE_SUB_NEW_FEEDS:
        for sub, posts in new_feed_cache.items():
            for p in posts:
                rid = safe_str(p.get("id"))
                if rid and rid not in candidates:
                    candidates[rid] = (p, f"new_{sub}")

    scanned_total = len(candidates)
    kept = 0

    for rid, (post, source) in candidates.items():
        if DEDUPE_WITH_EXISTING_CSV and (article_usid, rid) in existing_pairs:
            continue

        text = get_check_text(post)
        words = words_list(text)
        if len(words) > MAX_POST_WORDS:
            continue

        window_text = " ".join(words[:MAX_POST_WORDS])
        tokens = tokenize_set(window_text)

        matches = group_hit_count(tokens, groups)
        if matches < MIN_GROUP_MATCHES:
            continue

        matched_reps = []
        for g, rep in zip(groups, group_reps):
            if tokens & g:
                matched_reps.append(rep)

        rows.append({
            "article_usid": article_usid,
            "reddit_id": rid,
            "reddit_title": safe_str(post.get("title", "")),
            "reddit_selftext": safe_str(post.get("selftext", "")),
            "post_url": safe_str(post.get("url", "")),
            "reddit_permalink": "https://www.reddit.com" + safe_str(post.get("permalink", "")),
            "source": source,
            "checked_word_count": len(words),
            "groups_used": len(groups),
            "group_matches_in_window": matches,
            "matched_group_reps": ",".join(matched_reps),
            "query_de": query_de,
            "query_en": query_en,
            "saved_at_utc": datetime.now(timezone.utc).isoformat(),
        })
        kept += 1

    print(
        f"[{idx+1}/{len(df)}] {article_usid} | "
        f"groups_used={len(groups)}/{TARGET_GROUPS} | "
        f"scanned_total={scanned_total} | kept={kept}"
    )

# Write CSV
# Write to InfluxDB instead of CSV (DB-only)
from influx_io import write_orf_articles

if rows:
    written = write_orf_articles(rows)
    print(f"Wrote {written} rows to InfluxDB")
else:
    print("No matching posts found.")


Prefetched r/politics new: 800 posts
Prefetched r/austria new: 800 posts
Prefetched r/europe new: 800 posts
Prefetched r/worldnews new: 800 posts
Prefetched r/news new: 800 posts
[1/2] news:3416944 | groups_used=7/7 | scanned_total=5702 | kept=607
[2/2] news:3416938 | groups_used=7/7 | scanned_total=5778 | kept=515
Wrote 0 rows to InfluxDB


In [13]:
from influx_io import write_reddit_matches

written = write_reddit_matches(rows)
print("Wrote Reddit matches to InfluxDB:", written)


Wrote Reddit matches to InfluxDB: 1122


In [14]:
# --- Reddit write (CORRECT PLACE) ---
from influx_io import write_reddit_matches

reddit_rows = rows

# sanity check (optional)
if reddit_rows:
    print("Reddit row keys:", reddit_rows[0].keys())

written_reddit = write_reddit_matches(reddit_rows)
print("Wrote Reddit matches to InfluxDB:", written_reddit)


Reddit row keys: dict_keys(['article_usid', 'reddit_id', 'reddit_title', 'reddit_selftext', 'post_url', 'reddit_permalink', 'source', 'checked_word_count', 'groups_used', 'group_matches_in_window', 'matched_group_reps', 'query_de', 'query_en', 'saved_at_utc'])
Wrote Reddit matches to InfluxDB: 1122
